#  자연어 처리를 위한 NLTK와 KoNLPy 설치하기

## NLTK와 NLTK Data 설치
- 엔엘티케이(NLTK)는 자연어 처리를 위한 파이썬 패키지

In [22]:
# !pip install nltk

In [23]:
import nltk
nltk.__version__

'3.6.2'

In [24]:
# 해당 코드를 실행 후에 NLTK 실습에 필요한 각종 패키지와 코퍼스를 다운로드할 수 있다
#nltk.download()

## KoNLPY 설치
- 코엔엘파이(KoNLPy)는 한국어 자연어 처리를 위한 형태소 분석기 패키지

In [25]:
#!pip install konlpy

In [26]:
import konlpy
konlpy.__version__

'0.5.2'

### 한국어 문장분류기(KSS : Korean Sentence Splitter) 설치

In [27]:
#! pip install kss

In [28]:
import kss

# 텍스트 전처리
- 토큰화 : 자연어 처리에서 크롤링 등으로 얻어낸 코퍼스 데이터가 필요에 맞게 전처리되지 않은 상태라면, 해당 데이터를 사용하고자하는 용도에 맞게 토큰화(tokenization) & 정제(cleaning) & 정규화(normalization)하는 일을 하게된다. 이렇듯 주어진 코퍼스에서 토큰이라 불리는 단위로 나눈 작업을 토큰화 작업이라고 부른다.

# 품사 태깅(POS-tagging)

- 위에서 영어 문장에 대해서 토큰화를 수행하고, 이어서 품사 태깅을 수행하였다. Penn Treebank POG Tags에서 PRP는 인칭 대명사, VBP는 동사, RB는 부사, VBG는 현재부사, IN은 전치사, NNP는 고유 명사, NNS는 복수형 명사, CC는 접속사, DT는 관사를 의미한다.

- 각각의 명칭에 대한 줄임말은 다음과 같고, 그 외에 참고할 말한 것들은 다음  사이트에서 확인할 수 있다. [참고링크](https://bluebreeze.co.kr/1357)
    - PRP : personal pronoun 인칭 대명사(I, you, he, she)
    - VBP : verb, non-third person singular present 3인칭이 아닌 현재형 동사
    - RB : adverb	부사
    - VBG : verb, gerund or present 동명사 또는 현재분사(~ing)
    - IN : perposition, subordinating conjunction 전치사 종속 접속사


## KoNLPY 활용한 한글 토큰화 실습
- 한국어 자연어 처리를 위해서 KoNLPy를 주로 쓴다. KoNLPy를 통해 사용할 수 있는 형태소 분석기로 Okt(Open Korea Text), 메캅(Mecab), 코모란(Komoran), 한나눔(Hannanum), 꼬꼬마(Kkma)가 있다.
- 한국어 NLP에서 형태소 분석기를 사용한다는 것은 단어 토큰화가 아니라 정확히는 형태소(morpheme) 단위로 형태소 토큰화(morpheme tokenization)를 수행하게 됨을 뜻한다.

# Word2Vec

**Word2Vec의 하이퍼파라미터 값**

- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.  
- window = 컨텍스트 윈도우 크기  
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)  
- workers = 학습을 위한 프로세스 수  
- sg = 0은 CBOW, 1은 Skip-gram.  

## 한국어 데이터 다운로드 및 전처리

In [29]:
import urllib.request
from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
import os 

os.getcwd()

'/repo/course/sem21_01/youtube_summarizer/jupyter'

In [31]:
import json
import pandas as pd

path = '/repo/course/sem21_01/youtube_summarizer/dataset/article_dataset/train.jsonl'

### bfly train
sent_df = pd.read_json(path, lines=True, encoding="utf-8")
print('shape : {}'.format(sent_df.shape))
sent_df.head()

shape : (260697, 5)


,media,id,article_original,abstractive,extractive
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]"
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]"
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]"
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]"
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]"


In [32]:
texts = ' '.join([article[0] for article in sent_df['article_original'].values])

In [33]:
import re
# 영어 단어가 생각보다 많네?
eng = re.findall('([a-zA-Z]{1,})', texts)
len(eng)

109150

In [ ]:
# mecab - 방법1
# https://sosomemo.tistory.com/30
# Mecab설치
'''bash
cd /tmp
sudo wget https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
sudo tar xvf mecab-0.996-ko-0.9.2.tar.gz

cd /tmp/mecab-0.996-ko-0.9.2
sudo ./configure
sudo make check
sudo make install
'''

'''bash
cd /tmp
wget https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz
tar zxvf mecab-ko-dic-2.1.1-20180720.tar.gz

cd /tmp/mecab-ko-dic-2.1.1-20180720
sudo ./autogen.sh
sudo ./configure
sudo make
sudo make install
'''

'''bash
cd /tmp
git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
cd mecab-python-0.996
python3 setup.py build
python3 setup.py install
'''

'bash\ncd /tmp\ngit clone https://bitbucket.org/eunjeon/mecab-python-0.996.git\ncd mecab-python-0.996\npython3 setup.py build\npython3 setup.py install\n'

In [ ]:
# mecab 설치 - 방법1
# ! pip install konlpy
# ! sudo apt-get install curl git
# ! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.13).
git is already the newest version (1:2.17.1-1ubuntu0.8).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [34]:
from konlpy.tag import Okt  
from konlpy.tag import Mecab
mecab = Mecab()
#okt = Okt()

In [35]:
#stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','을','를','으로','자','에','와','한','하다', '합니다', '입니다']
stopwords = ['을', '를', '이', '가', '은', '는', '의', '에', '와', '으로', '합니다', '입니다']

In [36]:
import re

In [ ]:
tokenized_news_data = []
for news in sent_df['article_original']:
    for sentence in news:
        #print(f'previous : {sentence}')
        tmp_sen = re.sub('[^가-힣a-z]', ' ', sentence) # 영어 소문자와 한글을 제외한 모든 문자를 제거
        temp_X = mecab.morphs(tmp_sen) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        #print(f'after : {temp_X}')
        tokenized_news_data.append(temp_X)

In [ ]:
import pickle

pk_path = '/repo/course/sem21_01/youtube_summarizer/dataset/article_tokenized/tokenized_news_data.pkl'

# save pickle file
with open(pk_path, 'wb') as f:
    pickle.dump(tokenized_news_data, f)

In [20]:
import pickle

pk_path = '/repo/course/sem21_01/youtube_summarizer/dataset/article_tokenized/tokenized_news_data.pkl'

# load pickle file
with open(pk_path, 'rb') as f:
     tokenized_news_data = pickle.load(f)

In [ ]:
tokenized_news_data[0]

In [ ]:
sent_df['article_original'][0][0]

In [ ]:
tokenized_news_data[3]

In [ ]:
sent_df['article_original'][0]

### import pretrained model
**Pretraining 환경**
~~~
def train_word2vec(corpus_fname, model_fname):
    make_save_path(model_fname)
    corpus = Word2VecCorpus(corpus_fname)
    model = Word2Vec(corpus, size=100, workers=4, sg=1)
    model.save(model_fname)
~~~

In [ ]:
from gensim.models import Word2Vec

In [ ]:
import gensim
from gensim.models import Word2Vec

w2v_path = '/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/word2vec'

model = Word2Vec.load(w2v_path)

model


In [ ]:
model.wv.save_word2vec_format("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/word2vec.bin.gz", binary=False)

### finetune pretrained model

In [ ]:
# define new model
model_2 = Word2Vec(size=100, workers=4, sg=1)

# make voacb
model_2.build_vocab(tokenized_news_data)

In [ ]:
total_examples = model_2.corpus_count
print(total_examples)

In [ ]:
model_2.build_vocab([list(model.wv.vocab.keys())], update=True)
model_2.intersect_word2vec_format("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/word2vec.bin.gz", binary=False)


In [ ]:
model_2.train(tokenized_news_data, total_examples=total_examples, epochs=model_2.iter)

In [ ]:
# 어떤 단어들이 늘어났는가 확인

for k in model_2.wv.vocab:
    result = model.wv.vocab.get(k, "0")
    if result=="0":
        print(k, end=",")

In [ ]:
model_2.save("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/word2vec_finetuned.bin")

In [ ]:
# https://www.infoking.site/16
# https://jjdeeplearning.tistory.com/6

### Load Fine-tuned Word2Vec Model

In [ ]:
import gensim
from gensim.models import Word2Vec


model_prac = Word2Vec.load("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/word2vec_finetuned.bin")

model_prac

In [20]:
model_prac.wv.vectors("안녕")

TypeError: 'numpy.ndarray' object is not callable